In [1]:
!pip install transformers torch pandas -q
import pandas as pd
from transformers import pipeline


#DATA EXPLORATION

In [2]:
df=pd.read_csv('/content/SQuAD-v1.1.csv')

In [3]:
df.head()

,title,context,question,answer,answer_start,answer_end
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,541
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,213
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,296
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,420
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,126


In [4]:
df.tail()

,title,context,question,answer,answer_start,answer_end
87594,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon,229,235
87595,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon,414,421
87596,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk,476,481
87597,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975,199,203
87598,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What is KMC an initialism of?,Kathmandu Metropolitan City,0,27


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         87599 non-null  object
 1   context       87599 non-null  object
 2   question      87599 non-null  object
 3   answer        87596 non-null  object
 4   answer_start  87599 non-null  int64 
 5   answer_end    87599 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 4.0+ MB


In [6]:
df.iloc[0]['context']

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [7]:
df.iloc[0]['question']

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [8]:
df.iloc[0]['answer']

'Saint Bernadette Soubirous'

In [9]:
len(df)

87599

#choose pre trained model

In [10]:
from transformers import pipeline
qa = pipeline("question-answering",
              model="distilbert-base-cased-distilled-squad",
              tokenizer="distilbert-base-cased-distilled-squad")
qa


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
row_for_trial=df.iloc[:5]
predictions = []
for index, row in row_for_trial.iterrows():
  pred=qa(question=row['question'],context=row['context'])
  predictions.append(pred)

for i, pred in enumerate(predictions):
  print(f"Prediction for row {i}: {pred}")

for row in row_for_trial.iterrows():
  real=row_for_trial['answer']

print('REAL ANSWERS:',real)

Prediction for row 0: {'score': 0.996028007240966, 'start': 515, 'end': 541, 'answer': 'Saint Bernadette Soubirous'}
Prediction for row 1: {'score': 0.3604850471019745, 'start': 188, 'end': 213, 'answer': 'a copper statue of Christ'}
Prediction for row 2: {'score': 0.2535441508516669, 'start': 283, 'end': 296, 'answer': 'Main Building'}
Prediction for row 3: {'score': 0.9395833015441895, 'start': 381, 'end': 420, 'answer': 'a Marian place of prayer and reflection'}
Prediction for row 4: {'score': 0.37197253108024597, 'start': 92, 'end': 126, 'answer': 'a golden statue of the Virgin Mary'}
REAL ANSWERS: 0                 Saint Bernadette Soubirous
1                  a copper statue of Christ
2                          the Main Building
3    a Marian place of prayer and reflection
4         a golden statue of the Virgin Mary
Name: answer, dtype: object


In [12]:
import re
def _norm(s): return re.sub(r"[^a-zA-Z0-9أ-ي]+"," ",str(s).lower()).split()
def EM(p,g): return int(str(p).strip()==str(g).strip())
def F1(p,g):
    pt,gt=_norm(p),_norm(g)
    if not pt and not gt: return 1.0
    common=sum(min(pt.count(t),gt.count(t)) for t in set(pt))
    if common==0: return 0.0
    prec,rec=common/len(pt),common/len(gt); return 2*prec*rec/(prec+rec)


In [13]:
n = min(50, len(df))
ems, f1s = [], []
for i in range(n):
    r = df.iloc[i]
    p = qa(question=r["question"], context=r["context"])["answer"]
    ems.append(EM(p, r["answer"])); f1s.append(F1(p, r["answer"]))
(sum(ems)/len(ems), sum(f1s)/len(f1s))


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


(0.84, 0.9444761904761905)

#fine tunning

In [14]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tok = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.20, random_state=42, shuffle=True)
valid_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42, shuffle=True)
(len(train_df), len(valid_df), len(test_df))


(70079, 8760, 8760)

In [16]:
def prep(ex):
    enc=tok(ex["question"],ex["context"],truncation="only_second",max_length=384,padding="max_length",return_offsets_mapping=True)
    s,e=int(ex["answer_start"]),int(ex["answer_end"]); st=enc.char_to_token(0,s,sequence_index=1); en=enc.char_to_token(0,e-1,sequence_index=1)
    enc["start_positions"]=0 if st is None else st; enc["end_positions"]=0 if en is None else en
    return {k:enc[k] for k in["input_ids","attention_mask","start_positions","end_positions"]}


In [17]:
!pip install -q datasets
from datasets import Dataset
ds = Dataset.from_pandas(df); cols = ds.column_names
ds = ds.map(prep, remove_columns=cols); ds.set_format("torch")
from transformers import TrainingArguments, Trainer, default_data_collator, AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
args = TrainingArguments("qa-mini", per_device_train_batch_size=8, num_train_epochs=3, report_to="none")
trainer = Trainer(model=model, args=args, train_dataset=ds, data_collator=default_data_collator, tokenizer=tok)
trainer.train()


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4172503144.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, train_dataset=ds, data_collator=default_data_collator, tokenizer=tok)


Step,Training Loss
500,2.734600
1000,1.747100
1500,1.638900
2000,1.502200
2500,1.494400
3000,1.396500
3500,1.410200
4000,1.418600
4500,1.380000
5000,1.354500


TrainOutput(global_step=32850, training_loss=0.9095915749374227, metrics={'train_runtime': 11586.2606, 'train_samples_per_second': 22.682, 'train_steps_per_second': 2.835, 'total_flos': 2.575143204683213e+16, 'train_loss': 0.9095915749374227, 'epoch': 3.0})

In [18]:
from transformers import pipeline
qa_ft = pipeline("question-answering", model=model, tokenizer=tok)
out = qa_ft(question=r["question"], context=r["context"]); out


Device set to use cuda:0


{'score': 0.5262675285339355,
 'start': 0,
 'end': 21,
 'answer': 'Father Joseph Carrier'}

In [19]:
from transformers import pipeline
qa_val = pipeline("question-answering", model=trainer.model, tokenizer=tok, device=0)


Device set to use cuda:0


In [20]:
import re
def _norm(s): return re.sub(r"[^a-zA-Z0-9\u0660-\u0669أ-ي]+"," ",str(s).lower()).split()
def EM(p,g): return int(str(p).strip()==str(g).strip())
def F1(p,g): pt,gt=_norm(p),_norm(g); com=sum(min(pt.count(t),gt.count(t)) for t in set(pt)); return 1.0 if (not pt and not gt) else (0.0 if com==0 else 2*(com/len(pt))*(com/len(gt))/((com/len(pt))+(com/len(gt))))


In [21]:
n=min(100, len(valid_df)); em=f1=0.0
for i in range(n):
    r=valid_df.iloc[i]; a=qa_val(question=r["question"], context=r["context"])["answer"]
    em+=EM(a, r["answer"]); f1+=F1(a, r["answer"])
print("Valid EM,F1:", em/n, f1/n)


Valid EM,F1: 0.85 0.9512355889724309


In [22]:
bad=[]
for i in range(min(200,len(valid_df))):
    r=valid_df.iloc[i]; a=qa_val(question=r["question"], context=r["context"])["answer"]
    bad.append((F1(a,r["answer"]), r["question"], a, r["answer"]))
sorted(bad)[:5]


[(0.14285714285714288,
  'What was Kanye second arrest for?',
  'scuffle',
  'scuffle involving a photographer outside the famous Tup Tup Palace nightclub in Newcastle'),
 (0.33333333333333337,
  'What are these bars made out of?',
  'fiberglass',
  'fiberglass covered in wood laminate'),
 (0.33333333333333337,
  'What basic part of civilization has been characterized by achievement and challenges?',
  'sanitation',
  'Drinking water supply and sanitation'),
 (0.35294117647058826,
  "What did J.N. Sarkar say of Shivaji's nation building?",
  'the last great constructive genius and nation builder that the Hindu race has produced"',
  'great constructive genius'),
 (0.4,
  'Who else did they accuse of using child labor products?',
  'H&M and Zara',
  'Zara')]

In [23]:
qa_test = pipeline("question-answering", model=trainer.model, tokenizer=tok, device=0)
m=min(100, len(test_df)); EMt=F1t=0.0
for i in range(m):
    r=test_df.iloc[i]; a=qa_test(question=r["question"], context=r["context"])["answer"]
    EMt+=EM(a, r["answer"]); F1t+=F1(a, r["answer"])
print("Test EM,F1:", EMt/m, F1t/m)


Device set to use cuda:0


Test EM,F1: 0.82 0.9397811384521911


In [24]:
trainer.save_model("qa-final")
tok.save_pretrained("qa-final")


('qa-final/tokenizer_config.json',
 'qa-final/special_tokens_map.json',
 'qa-final/vocab.txt',
 'qa-final/added_tokens.json',
 'qa-final/tokenizer.json')